In [93]:
%run C:/Users/MohammedSB/Desktop/projects/Hypertension/Requirements.ipynb

In [94]:
from collections import Counter

In [7]:
PATH = r"D:\\MohammedSB\\Fundus\\"
CSV_PATH = {"HTNPath": PATH + r"HTN", "NonHTNPath": PATH + "NonHTN"}

MODELS_PATH = r"C:\Users\MohammedSB\Desktop\projects\Hypertension\models\\"

In [96]:
htn = pd.read_excel(CSV_PATH["HTNPath"] + r"\HTN_Age_Gender.xlsx")
htn["Hypertension"] = 1

# Clean Hypertension
htn.columns = htn.columns.str.strip()
htn["Eye"] = htn["Eye"].str.strip()

# Add "File Path" column
htn["Image Path"] = None
addFilePath(htn, os.listdir(CSV_PATH["HTNPath"]))

# non-Hypertension
non_htn = pd.read_excel(CSV_PATH["NonHTNPath"] + r"\\NonHTN_Age_Gender.xlsx")
non_htn["Hypertension"] = 0

# Clean non-Hypertension
non_htn.columns = non_htn.columns.str.strip()
non_htn["Eye"] = non_htn["Eye"].str.strip()

# Add "Image Path" column
non_htn["Image Path"] = None
addFilePath(non_htn, os.listdir(CSV_PATH["NonHTNPath"]))

# Add the two df's
df = pd.concat([htn, non_htn]) 

# Clean again
df = df.dropna()
df["Gender"] = df["Gender"].str.strip()
df.reset_index(inplace=True, drop=True)

In [97]:
df

,MRN,Age,Gender,Exam Date,Eye,Hypertension,Image Path
0,10378,67.0,M,2020-06-07,OS,1,10378_11052020_162440_Color_L_001.jpg
1,18310,60.0,F,2020-09-29,OD,1,18310_12092020_160912_Color_R_001.jpg
2,18310,60.0,F,2020-09-29,OS,1,18310_12092020_160923_Color_L_001.jpg
3,19616,66.0,M,2020-08-26,OD,1,19616_12102020_120540_Color_R_001.jpg
4,19616,66.0,M,2020-08-26,OS,1,19616_12102020_120557_Color_L_001.jpg
...,...,...,...,...,...,...,...
1138,2915822,25.0,M,2021-01-21,OD,0,2915822_07142021_160104_Color_R_001.jpg
1139,2915822,25.0,M,2021-01-21,OS,0,2915822_07142021_160117_Color_L_001.jpg
1140,2917878,55.0,M,2021-02-08,OD,0,2917878_07142021_160559_Color_R_001.jpg
1141,2917878,55.0,M,2021-02-08,OS,0,2917878_07142021_160612_Color_L_001.jpg


In [76]:
df[df["Hypertension"] == 1]["Gender"].value_counts()

F    297
M    260
Name: Gender, dtype: int64

In [25]:
(df.pivot_table(columns=['MRN'], aggfunc='size') == 2).sum()

517

In [35]:
htn_clean = df[df["Hypertension"] == 1]
non_htn_clean = df[df["Hypertension"] == 0]

In [92]:
df[df["Gender"] == "M"]["Age"].describe()

count    540.000000
mean      50.296296
std       21.560571
min        6.000000
25%       34.000000
50%       57.000000
75%       67.000000
max       85.000000
Name: Age, dtype: float64

In [68]:
shapes = []

In [69]:
for image in htn_clean["Image Path"]:
    im = cv2.imread(CSV_PATH["HTNPath"] + f"/{image}")
    shapes.append(im.shape)

In [70]:
for image in non_htn_clean["Image Path"]:
    im = cv2.imread(CSV_PATH["NonHTNPath"] + f"/{image}")
    shapes.append(im.shape)

In [72]:
Counter(shapes)

Counter({(1024, 1024, 3): 1143})

In [22]:
image_size = 500

transform = T.Compose([
    T.Resize((image_size, image_size)),
    T.ToTensor(),
])


images = ImageFolder(PATH, transform=transform)

In [18]:
loader = DataLoader(images, batch_size=1)

In [25]:
psum    = torch.tensor([0.0, 0.0, 0.0])
psum_sq = torch.tensor([0.0, 0.0, 0.0])

for inputs, _ in tqdm(loader):
    psum    += inputs.sum(axis        = [0, 2, 3])
    psum_sq += (inputs ** 2).sum(axis = [0, 2, 3])
    

count = len(loader.dataset) * image_size * image_size

# mean and std
total_mean = psum / count
total_var  = (psum_sq / count) - (total_mean ** 2)
total_std  = torch.sqrt(total_var)

100%|██████████| 1170/1170 [00:23<00:00, 49.09it/s]


In [26]:
total_mean

tensor([0.3675, 0.1760, 0.1047])

In [27]:
total_std

tensor([0.2289, 0.1180, 0.0748])